In [30]:
__author__ = 'stephanie'

#Demo Created and run against little bear river demo database. scripts are located at 
#https://github.com/ODM2/ODM2/tree/master/usecases/littlebearriver/sampledatabases

import matplotlib.pyplot as plt
from matplotlib import dates
%matplotlib notebook
# inline

from odm2api.ODMconnection import dbconnection
from odm2api.ODM2.services.readService import *


# Create a connection to the ODM2 database
# ----------------------------------------

#createconnection (dbtype, servername, dbname, username, password)
session_factory = dbconnection.createConnection('mysql', 'localhost', 'ODM2', 'ODM', 'odm')#mysql
#session_factory = dbconnection.createConnection('connection type: sqlite|mysql|mssql|postgresql', '/your/path/to/db/goes/here', 2.0)#sqlite
# session_factory= dbconnection.createConnection('mssql', "(local)", "LBRODM2", "ODM", "odm")#win MSSQL
# session_factory= dbconnection.createConnection('mssql', "odm2", "", "ODM", "odm")#mac/linux MSSQL

In [31]:
_session = session_factory.getSession()

read = ReadODM2(session_factory)

In [32]:
# Run some basic sample queries.
# ------------------------------
# Get all of the variables from the database and print their names to the console
allVars = read.getVariables()

for x in allVars:
    print x.VariableCode + ": " + x.VariableNameCV

In [9]:
# Get all of the people from the database
allPeople = read.getPeople()

for x in allPeople:
    print x.PersonFirstName + " " + x.PersonLastName

try:
    print "\n-------- Information about an Affiliation ---------"
    allaff = read.getAffiliations()
    for x in allaff:
        print x.PersonObj.PersonFirstName + ": " + str(x.OrganizationID)
except Exception as e:
    print "Unable to demo getAllAffiliations", e

Jeff Horsburgh
Nancy Mesner
Amber Spackman
Erin Jones
Janet Barnette
Jason Bahr
Chris Cox
Unknown Person
Lisa Ward

-------- Information about an Affiliation ---------
Jeff: 1
Jeff: 2
Amber: 3
Nancy: 4
Amber: 6
Jeff: 6
Erin: 46
Janet: 48
Jason: 45
Chris: 45
Unknown: 45
Lisa: 47


In [33]:
# Get all of the SamplingFeatures from the database that are Sites

try:
    siteFeatures = read.getSamplingFeatures(type='Site')
    numSites = len(siteFeatures)
    print "\n-------- Information about an SamplingFeature of type 'Site'---------"
    for x in siteFeatures:
        print x.SamplingFeatureCode + ": " + x.SamplingFeatureName
except Exception as e:
    print "Unable to demo getSamplingFeaturesByType", e


# Now get the SamplingFeature object for a SamplingFeature code
try:
    sf = read.getSamplingFeatures(codes= ['USU-LBR-Mendon'])[0]
    
    print "\n-------- Information about an individual SamplingFeature ---------"
    print "The following are some of the attributes of a SamplingFeature retrieved using getSamplingFeatureByCode(): \n"
    print "SamplingFeatureCode: " + sf.SamplingFeatureCode
    print "SamplingFeatureName: " + sf.SamplingFeatureName
    print "SamplingFeatureDescription: %s" % sf.SamplingFeatureDescription
    print "SamplingFeatureGeotypeCV: %s" % sf.SamplingFeatureGeotypeCV
    print "SamplingFeatureGeometry: %s" % sf.FeatureGeometry
    print "Elevation_m: %s" % str(sf.Elevation_m)
except Exception as e:
    print "Unable to demo getSamplingFeatureByCode: ", e

In [14]:
#add sampling feature
print "\n------------ Create Sampling Feature --------- \n",
try:
    from odm2api.ODM2.models import SamplingFeatures
    newsf = SamplingFeatures()
    session = session_factory.getSession()
    newsf.FeatureGeometry = "POINT(-111.946 41.718)"
    newsf.Elevation_m=100
    newsf.ElevationDatumCV=sf.ElevationDatumCV
    newsf.SamplingFeatureCode= "TestSF"
    newsf.SamplingFeatureDescription = "this is a test to add Feature Geomotry"
    newsf.SamplingFeatureGeotypeCV= "Point"
    newsf.SamplingFeatureTypeCV=sf.SamplingFeatureTypeCV
    newsf.SamplingFeatureUUID= sf.SamplingFeatureUUID+"2"
    session.add(newsf)
    #session.commit()
    print "new sampling feature added to database", newsf

except Exception as e :
    print "error adding a sampling feature: " + str(e)

In [34]:
# Drill down and get objects linked by foreign keys
print "\n------------ Foreign Key Example --------- \n",
try:
    # Call getResults, but return only the first result
    firstResult = read.getResult()[0]
    print "The FeatureAction object for the Result is: ", firstResult.FeatureActionObj
    print "The Action object for the Result is: ", firstResult.FeatureActionObj.ActionObj
    print ("\nThe following are some of the attributes for the Action that created the Result: \n" +
           "ActionTypeCV: " + firstResult.FeatureActionObj.ActionObj.ActionTypeCV + "\n" +
           "ActionDescription: " + firstResult.FeatureActionObj.ActionObj.ActionDescription + "\n" +
           "BeginDateTime: " + str(firstResult.FeatureActionObj.ActionObj.BeginDateTime) + "\n" +
           "EndDateTime: " + str(firstResult.FeatureActionObj.ActionObj.EndDateTime) + "\n" +
           "MethodName: " + firstResult.FeatureActionObj.ActionObj.MethodObj.MethodName + "\n" +
           "MethodDescription: " + firstResult.FeatureActionObj.ActionObj.MethodObj.MethodDescription)
except Exception as e:
    print "Unable to demo Foreign Key Example: ", e

In [28]:
# Now get a particular Result using a ResultID
print "\n------- Example of Retrieving Attributes of a Time Series Result -------"
try:
    #tsResult = read.getTimeSeriesResultByResultId(1)
    tsResult = read.getResults(ids = [1])[0]
    print (
        "The following are some of the attributes for the TimeSeriesResult retrieved using getTimeSeriesResultByResultID(): \n" +
        "ResultTypeCV: " + tsResult.ResultTypeCV + "\n" +
        # Get the ProcessingLevel from the TimeSeriesResult's ProcessingLevel object
        "ProcessingLevel: " + tsResult.ProcessingLevelObj.Definition + "\n" +
        "SampledMedium: " + tsResult.SampledMediumCV + "\n" +
        # Get the variable information from the TimeSeriesResult's Variable object
        "Variable: " + tsResult.VariableObj.VariableCode + ": " + tsResult.VariableObj.VariableNameCV + "\n"
                                                                                                        "AggregationStatistic: " + tsResult.AggregationStatisticCV + "\n" +
        "Elevation_m: " + str(sf.Elevation_m) + "\n" +
        # Get the site information by drilling down
        "SamplingFeature: " + tsResult.FeatureActionObj.SamplingFeatureObj.SamplingFeatureCode + " - " +
        tsResult.FeatureActionObj.SamplingFeatureObj.SamplingFeatureName)
except Exception as e:
    print "Unable to demo Example of retrieving Attributes of a time Series Result: ", e


------- Example of Retrieving Attributes of a Time Series Result -------
The following are some of the attributes for the TimeSeriesResult retrieved using getTimeSeriesResultByResultID(): 
ResultTypeCV: Time series coverage
ProcessingLevel: Raw data
SampledMedium: Not applicable
Variable: USU3: Battery voltage
AggregationStatistic: Minimum
Elevation_m: 1345.0
SamplingFeature: USU-LBR-Mendon - Little Bear River at Mendon Road near Mendon, Utah


In [27]:
# Get the values for a particular TimeSeriesResult
print "\n-------- Example of Retrieving Time Series Result Values ---------"


tsValues = read.getResultValues(resultid =1)  # Return type is a pandas dataframe
# Print a few Time Series Values to the console
try:
    print tsValues.head()
except Exception as e:
    print e

# Plot the time series

try:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    tsValues.plot(x='ValueDateTime', y='DataValue', kind='line',
                  title=tsResult.VariableObj.VariableNameCV + " at " + tsResult.FeatureActionObj.SamplingFeatureObj.SamplingFeatureName,
                  ax=ax)
    ax.set_ylabel(tsResult.VariableObj.VariableNameCV + " (" + tsResult.UnitsObj.UnitsAbbreviation + ")")
    ax.set_xlabel("Date/Time")
    ax.xaxis.set_minor_locator(dates.MonthLocator())
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(dates.YearLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n%Y'))
    ax.grid(True)
    plt.show()
except Exception as e:
    print "Unable to demo plotting of tsValues: ", e